## 0. Устанавливаем библиотеки и модель.

Ссылка на модель: https://huggingface.co/Babelscape/wikineural-multilingual-ner?text=My+name+is+Wolfgang+and+I+live+in+Berlin.



In [1]:
!pip3 install pymorphy2
!pip3 install razdel
!pip3 install nltk
!pip3 install transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.5/55.5 kB 1.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.2/8.2 MB 64.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for docopt: filename=docopt-0.6.2-py2.py3-none-any.whl size=13706 sha256=dbb227c0a2ed0636ffb1dc41e0ef9385c1fb0e6faa7760f1eac527f163194e65
  Stored in directory: /root/.cache/pip/wheels/fc/ab/d4/5da2067ac95b36618c629a5f93f809425700506f72c9732fac
Successfully built docopt


In [8]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
from glob import glob
from typing import List, Dict, Optional, Union, Tuple
import re
import pymorphy2
import string
import razdel
import numpy as np
from nltk.corpus import stopwords
import pandas as pd
import nltk
import torch.nn.functional as F

from torch import Tensor
from transformers import AutoTokenizer, AutoModel

from sklearn.metrics import ndcg_score
nltk.download('stopwords')
string.punctuation

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

In [3]:
from transformers import AutoTokenizer, AutoModelForTokenClassification
from transformers import pipeline

tokenizer = AutoTokenizer.from_pretrained("Babelscape/wikineural-multilingual-ner")
model = AutoModelForTokenClassification.from_pretrained("Babelscape/wikineural-multilingual-ner")

nlp = pipeline("ner", model=model, tokenizer=tokenizer, grouped_entities=True)
example = "My name is Wolfgang and I live in Berlin"

ner_results = nlp(example)
print(ner_results)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/333 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/996k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.96M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.19k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/709M [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/pipelines/token_classification.py:168: UserWarning: `grouped_entities` is deprecated and will be removed in version v5.0.0, defaulted to `aggregation_strategy="simple"` instead.
  warnings.warn(


[{'entity_group': 'PER', 'score': 0.96688974, 'word': 'Wolfgang', 'start': 11, 'end': 19}, {'entity_group': 'LOC', 'score': 0.9996282, 'word': 'Berlin', 'start': 34, 'end': 40}]


In [41]:
ner_results = nlp('Меня зовут Дима, и я живу в Москве')
print(ner_results)

[{'entity_group': 'PER', 'score': 0.8659103, 'word': 'Дима', 'start': 11, 'end': 15}, {'entity_group': 'LOC', 'score': 0.99795604, 'word': 'Москве', 'start': 28, 'end': 34}]


## 1. Читаем данные

Ссылка на статью: https://ru.wikipedia.org/wiki/%D0%94%D0%B6%D0%B5%D1%80%D0%B0%D1%80%D0%B4,_%D0%94%D0%B6%D0%BE%D0%BD_(%D0%B8%D0%B5%D0%B7%D1%83%D0%B8%D1%82)

In [97]:
class TooLongSentence(Exception):
  pass

class TextPreparer:
  raw_text: str = ''
  prepared_text: str = ''
  raw_sentences: List[str] = []

  def __init__(self):
    pass

  # Читаем тексты из Gdrive
  def read_texts(self, text_names: Optional[List[str]] = None):
    texts = []
    if not text_names:
      text_names = sorted(glob('drive/MyDrive/texts/*'))
    for tname in text_names:
      with open(tname, 'r') as r:
        texts.append(r.read())
    self.raw_text = '\n'.join(texts)

  # Базово предобрабатываем тексты регулярками, избавляемся от лишних пробелов и ссылок
  # Upd. не делаем, т.к. в ответе нужны точные места использования
  def clean_and_split_text(self):
    pass
    # self.prepared_text = re.sub(r'\[\d+\]', '', self.raw_text)
    # self.prepared_text = re.sub(r'англ\.', '', self.prepared_text)
    # self.prepared_text = re.sub(r'\s+', ' ', self.prepared_text)

  # С помощью PyMorphy приводим все слова к каноничному виду (по возможности)
  def prepare_sentences(self):
    self.raw_sentences = re.split(r'[!\?\.]', self.raw_text)
    self.raw_sentences = [sn + '.' for sn in self.raw_sentences] # Возвращаем правильное позиционирование

    max_sample_size = 400
    self.samples = []
    sample = ''
    i = 0
    while i != len(self.raw_sentences):
      while len(sample) < max_sample_size and i != len(self.raw_sentences):
        sentence = self.raw_sentences[i]
        if len(sentence) >= max_sample_size:
          raise TooLongSentence(f"Sentence size is bigger than example size: {len(sentence)}")
        if len(sample) + len(sentence) >= max_sample_size:
          break
        sample = ''.join([sample, sentence])
        i += 1
      self.samples.append(sample)
      sample = ''

In [98]:
t = TextPreparer()
t.read_texts()
t.clean_and_split_text()
t.prepare_sentences()

In [100]:
t.samples

['Джон Джерард (англ. John Gerard; 4 октября 1564, предположительно Нью-Брин, Дербишир, Королевство Англия — 27 июля 1637, Рим, Папская область) — английский иезуит, долгое время занимавшийся подпольной миссионерской деятельностью на территории Англии. Был схвачен, но выдержал пытки и смог бежать.',
 ' Только после раскрытия Порохового заговора в 1605 году, когда продолжение миссии стало невозможным, Джерард уехал на континент. Он написал ряд богословских трудов и автобиографию, ценную для историков. В мини-сериале «Порох» (2017) Джерарда сыграл Роберт Эммс.',
 '\n\nДжон Джерард был вторым сыном сэра Томаса Джерарда из Брина, Ланкашир[4], и его жены Элизабет Порт, дочери и наследницы сэра Джона Порта из Этволла, Дербишир. Эта семья отличалась симпатиями к католичеству, которое подвергалось в Англии преследованиям. Джерард родился предположительно в Нью-Брине 4 октября 1564 года. Он учился в английском колледже в Дуэ, куда приехал 29 августа 1577 года.',
 ' В марте следующего года, по-в

## 2. Применяем к предложениям нашу модельку

In [101]:
ner_result = nlp(t.samples)

In [102]:
ner_result

[[{'entity_group': 'PER',
   'score': 0.9655545,
   'word': 'Джон Джерард',
   'start': 0,
   'end': 12},
  {'entity_group': 'PER',
   'score': 0.99418783,
   'word': 'John Gerard',
   'start': 20,
   'end': 31},
  {'entity_group': 'LOC',
   'score': 0.9993449,
   'word': 'Нью - Брин',
   'start': 66,
   'end': 74},
  {'entity_group': 'LOC',
   'score': 0.9991207,
   'word': 'Дербишир',
   'start': 76,
   'end': 84},
  {'entity_group': 'LOC',
   'score': 0.9944261,
   'word': 'Королевство Англия',
   'start': 86,
   'end': 104},
  {'entity_group': 'LOC',
   'score': 0.999637,
   'word': 'Рим',
   'start': 121,
   'end': 124},
  {'entity_group': 'LOC',
   'score': 0.99742454,
   'word': 'Папская область',
   'start': 126,
   'end': 141},
  {'entity_group': 'LOC',
   'score': 0.9988562,
   'word': 'Англии',
   'start': 243,
   'end': 249}],
 [{'entity_group': 'MISC',
   'score': 0.92900217,
   'word': 'Порохового заговора',
   'start': 24,
   'end': 43},
  {'entity_group': 'PER',
   'sco

In [103]:
#№ Сформируем частотный список слов
total_entities = 0
word_appearances = {}
for sentence_result in ner_result:
  total_entities += len(sentence_result)
  for entity in sentence_result:
    word = entity['word']
    if word not in word_appearances:
      word_appearances[word] = 0
    word_appearances[word] += 1
for key in word_appearances:
    word_appearances[key] /= total_entities

import json
json.dump(word_appearances, open('drive/MyDrive/colab_results/frequency.json', 'w'), indent=1)

In [104]:
## Подправим смещения сущностей из результата NER модели под текст
import copy
offset = 0
ner_prepared = []
for i, sentence_result in enumerate(ner_result):
  for entity in sentence_result:
    ner_prepared.append(copy.deepcopy(entity))
    ner_prepared[-1]['score'] = float(ner_prepared[-1]['score'])
    ner_prepared[-1]['start'] += offset
    ner_prepared[-1]['end'] += offset
  offset += len(t.samples[i])
json.dump(ner_prepared, open('drive/MyDrive/colab_results/entities.json', 'w'), indent=1)

In [105]:
word_appearances

{'Джон Джерард': 0.030303030303030304,
 'John Gerard': 0.015151515151515152,
 'Нью - Брин': 0.015151515151515152,
 'Дербишир': 0.030303030303030304,
 'Королевство Англия': 0.015151515151515152,
 'Рим': 0.045454545454545456,
 'Папская область': 0.015151515151515152,
 'Англии': 0.030303030303030304,
 'Порохового заговора': 0.030303030303030304,
 'Джерард': 0.19696969696969696,
 '« Порох »': 0.015151515151515152,
 'Джерарда': 0.045454545454545456,
 'Роберт Эммс': 0.015151515151515152,
 'Томаса Джерарда': 0.015151515151515152,
 'Брина': 0.015151515151515152,
 'Ланкашир': 0.015151515151515152,
 'Элизабет Порт': 0.015151515151515152,
 'Джона Порта': 0.015151515151515152,
 'Этволла': 0.015151515151515152,
 'Джера': 0.015151515151515152,
 'Нью - Брине': 0.015151515151515152,
 'Дуэ': 0.015151515151515152,
 'Реймс': 0.015151515151515152,
 'Эксетер - колледж': 0.015151515151515152,
 'Оксфорде': 0.015151515151515152,
 'Клермонском колледже': 0.015151515151515152,
 'Париже': 0.015151515151515152,
 

In [106]:
ner_prepared

[{'entity_group': 'PER',
  'score': 0.9655544757843018,
  'word': 'Джон Джерард',
  'start': 0,
  'end': 12},
 {'entity_group': 'PER',
  'score': 0.9941878318786621,
  'word': 'John Gerard',
  'start': 20,
  'end': 31},
 {'entity_group': 'LOC',
  'score': 0.9993448853492737,
  'word': 'Нью - Брин',
  'start': 66,
  'end': 74},
 {'entity_group': 'LOC',
  'score': 0.9991207122802734,
  'word': 'Дербишир',
  'start': 76,
  'end': 84},
 {'entity_group': 'LOC',
  'score': 0.9944260716438293,
  'word': 'Королевство Англия',
  'start': 86,
  'end': 104},
 {'entity_group': 'LOC',
  'score': 0.9996370077133179,
  'word': 'Рим',
  'start': 121,
  'end': 124},
 {'entity_group': 'LOC',
  'score': 0.9974245429039001,
  'word': 'Папская область',
  'start': 126,
  'end': 141},
 {'entity_group': 'LOC',
  'score': 0.9988561868667603,
  'word': 'Англии',
  'start': 243,
  'end': 249},
 {'entity_group': 'MISC',
  'score': 0.9290021657943726,
  'word': 'Порохового заговора',
  'start': 320,
  'end': 339}

In [108]:
t.raw_text[3676:3682]

'Anglia'